In [121]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlencode
import csv

In [122]:
mug_urls = [
    "https://www.amazon.com/YETI-Rambler-Stainless-Insulated-MagSlider/dp/B073WJMKHN/",
    "https://www.amazon.com/Contigo-AUTOSEAL-Vaccuum-Insulated-Stainless-Travel/dp/B01JIEIY0G",
    "https://www.amazon.com/dp/B07L6LL95P",
    "https://www.amazon.com/Thermos-Stainless-Ounce-Travel-Steel/dp/B01DZQT0E2",
    "https://www.amazon.com/THERMOS-Stainless-Travel-Ounce-Matte/dp/B08JWMVB43",
    "https://www.amazon.com/Contigo-SnapSeal-Vacuum-Insulated-Stainless-Gunmetal/dp/B00IR77HMW",
    "https://www.amazon.com/Contigo-AUTOSEAL-Transit-Stainless-Travel/dp/B0160LEYFY"
]

In [123]:
washing_powder_urls = [
    "https://www.amazon.com/Arm-Hammer-Super-Washing-Soda/dp/B0029XNTEU",
    "https://www.amazon.com/Gain-Laundry-Detergent-Regular-Moonlight/dp/B09DHVK97X",
    "https://www.amazon.com/Tide-Powder-Laundry-Detergent-Mountain/dp/B00BZWUJM6",
    "https://www.amazon.com/Ariel-Powder-Laundry-Detergent-Freshness/dp/B0BJMBBQJT",
    "https://www.amazon.com/OxiClean-Odor-Blasters-80-Ounce/dp/B01BUNHFAS",
    "https://www.amazon.com/Ariel-Ultra-Powder-Laundry-Detergent/dp/B0BJNCKHD8",
    "https://www.amazon.com/Gain-Freshlock-Island-Powder-Detergent/dp/B009P7W886",
    "https://www.amazon.com/Forever-New-Granular-Natural-Detergent/dp/B002USD5FC"
    "https://www.amazon.com/Mollys-Suds-Original-Detergent-Sensitive/dp/B0073LGECW"
]

In [124]:
dishwashing_tablets_urls = [
    "https://www.amazon.com/Finish-Dishwasher-Detergent-Powerball-Dishwashing/dp/B06XX33G82",
    "https://www.amazon.com/Cascade-Platinum-ActionPacs-Dishwasher-Detergent/dp/B086ZJTVS5",
    "https://www.amazon.com/Finish-Technology-Dishwasher-Detergent-Dishwashing/dp/B0857XKM2M",
    "https://www.amazon.com/Seventh-Generation-Fragrance-Dishwasher-Detergent/dp/B01FHUJ1FC",
    "https://www.amazon.com/Palmolive-PowerPacs-Dishwasher-Detergent-Biodegradable/dp/B08L8MZNJ2",
    "https://www.amazon.com/Planet-Automatic-Clear-Dishwasher-Ounce/dp/B01KLFR5V0",
    "https://www.amazon.com/Ecover-Automatic-Dishwasher-Tablets-Citrus/dp/B00MFPZJZ4",
    "https://www.amazon.com/Cascade-Complete-ActionPacs-Dishwasher-Detergent/dp/B01NCJSM2T",
    "https://www.amazon.com/Finish-Technology-Dishwasher-Detergent-Dishwashing/dp/B0857WT8R9",
    "https://www.amazon.com/Ecover-Automatic-Dishwasher-Tablets-Fragrance-Free/dp/B0080L99GC",
    "https://www.amazon.com/Seventh-Generation-Dishwasher-Detergent-Citrus/dp/B01FHUJ1ZW",
    "https://www.amazon.com/Finish-Gelpacs-Dishwasher-Detergent-Tablets/dp/B002ODC2DE"
]

In [125]:
urls = [mug_urls, washing_powder_urls, dishwashing_tablets_urls]
categories = ["mug", "washing powder", "dishwashing tablet"]
star_filter_dict = {1: "one_star", 2: "two_star", 3: "three_star", 4: "four_star", 5: "five_star"}

In [126]:
reviews_dict = {
    "mug": [],
    "washing powder": [],
    "dishwashing tablet": []
}

sentiments_dict = {
    "mug": [],
    "washing powder": [],
    "dishwashing tablet": []
}

In [128]:
api_key = ""
stars = 3
review_limit = 5

data_string = ""

for index, url_set in enumerate(urls):
    i = 0
    for url in url_set:
        page = 1
        while page != -1 and i <= review_limit:
            url = url.replace("/dp/", "/product-reviews/")
            url_params = url + '/?filterByStar=' + star_filter_dict.get(stars) + '&pageNumber=' + str(page)

            params = {"api_key": api_key, "url": url_params}
            response = requests.get("http://api.scraperapi.com/",
                                      params=urlencode(params))
            soup = BeautifulSoup(response.text, 'html.parser')

            for item in soup.find_all("span", {"data-hook": "review-body"}):
                data_string = data_string + item.get_text()
                reviews_dict[categories[index]].append(data_string)
                data_string = ""
                i += 1

            page = page + 1 if soup.find_all('li', class_="a-last") != [] else -1

In [129]:
for category in reviews_dict:
    for review in reviews_dict[category]:
        if stars in {5, 4}:
            sentiments_dict[category].append("positive")
        elif stars == 3:
            sentiments_dict[category].append("neutral")
        else:
            sentiments_dict[category].append("negative") 

In [130]:
for category in categories:
    paired_reviews = {"reviews": reviews_dict[category], "sentiment": sentiments_dict[category]}

    df = pd.DataFrame.from_dict(paired_reviews, orient="index")
    df.dropna(axis=1, inplace=True)

    final_df = df.T
    final_df["reviews"] = final_df["reviews"].str.replace('\n','')
    final_df["product_category"] = category
    final_df.to_csv(category + "_reviews.csv", index=False, header=True)